# Noms du binome: Calamy, Mauron

# Prénoms du binome: Victor, Pierre

## Note: exporter le compte rendu basé sur le notebook au format pdf


# Entropie et génération de mots de passe
* L'objectif de ce tp est de continuer à se familiariser avec la notion d'entropie, mais aussi de comprendre le lien qu'il existe entre cette mesure informationelle et la sécurité d'un générateur (humain ou executable) de mots de passes
* Ainsi, nous proposons d'étudier l'entropie d'un tel générateur, et ce en fonction du modèle probabiliste considéré pour le modéliser (contruit à partir d'une lettre, de deux lettres, de 4 lettres, ...). A l'aide de tirrages aléatoires, nous estimerons également le temps moyen nécessaire pour trouver un mot de passe à partir de ce modèle.
* A la fin de ce TP, nous considérerons un générateur de mots de passe spécifique qui générera un mot de passe en prennant **8 lettres consécutives dans un texte** (sans se soucier des espaces). Ces lettres peuvent faire parti d'un ou de plusieurs mots consécutifs.
* Nous faisons l'hypothèse que le texte n'est composé que des 26 lettres de l'alphabet, sans majuscules ni accents

Nous chercherons aussi à comprendre (voir dernière question):
- les bonnes pratiques pour le défenseur, i.e. la personne cherchant à générer/construire un système de génération de mots de passe.
- les bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe.

**Il est important de commenter vos réponses, en utilisant des cellules markdown**


In [1]:
import numpy as np

from numpy import genfromtxt
from pandas import read_csv
import pandas as pd
import time

#### Modèle monogramme (une lettre)
* On récupére des données composées de [lettre,frequence d'apparition de la lettre] (voir fichier csv pour [comma-separated-value](https://en.wikipedia.org/wiki/Comma-separated_values)) 

In [2]:
monogramme = read_csv('monogramme.csv')
freq_mono = (monogramme['frequency']).values
letters_mono = (monogramme['letters']).values



Q: Quelles sont les 5 lettres les plus représentées ?

In [7]:
# TO DO

print(letters_mono[:5])


['E' 'S' 'A' 'N' 'T']


Les 5 lettres les plus représentées sont : E S A N T dans cet ordre.

Ecrire une fonction qui calcule l'entropie à partir d'un vecteur constitué de probabilités empiriques (note, il est important de bien "gérer le cas ou la probabilité est nulle).

In [23]:
def entropie(freq):
    ent = 0
    for f in freq :
        if (f!=0):
            ent -= f*np.log2(f)
    return ent

Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [24]:
entropie_mono = 8*entropie(freq_mono)
print(entropie_mono)

31.676242429778338


L'entropie d'un mot de passe de 8 lettres est de quasiment 32 bits.

Q: A l'aide de la fonction `np.random.choice()`, estimer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ? (note: ici le tirage n'est pas forcemment réaliste, car aléatoire, mais l'idée est surtout de mesurer le temps minimal nécessaire pour générer N mots de passes).

In [19]:
t = time.time()


mdps = np.random.choice(letters_mono,size=(100000,8),p=freq_mono)

t_100000 = time.time() - t
print(t_100000, 's')


0.07190775871276855 s


Générer 100 000 mots de passes de 8 lettres dure 0.072 secondes... c'est assez rapide

Nous definissons l'"entropie du devin" G (guessing entropie) comme le **nombre moyen d'essais successif nécessaires pour trouver un mot de passe à partir de notre générateur**.
    On peut montrer que $G\geq 2^H/4+1$ où $H$ est l'entropie de la source (voir le papier Password_Entropy_and_Password_Quality.pdf )

Q: calculer le minorant de $G$ pour ce modèle

In [26]:
G_min = 2**entropie_mono/4 + 1
print(G_min)

857904864.6814479


Q: combien de temps cela prendra-t-il pour générer un mot de passe si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [31]:
print("La génération d'un bon mot de passe prend en moyenne basse", G_min*t_100000/6000000, "minutes")

La génération d'un bon mot de passe prend en moyenne basse 10.281669334670651 minutes


On propose maintenant d'utiliser un modèle plus évolué qui est construit à partir de la probabilité conjointe de deux lettres successives (bigramme)

In [33]:

bigramme = read_csv('bigramme.csv',keep_default_na=False)


#print(bigramme.head(5))
freq_bi = (bigramme['frequency']).values

letters_bi = (bigramme['letters']).values



Q: Quelles sont les 5 couples de lettres les plus représentés ?

In [37]:
freq_bi_sort = np.argsort(freq_bi)[-5:]
print(letters_bi[freq_bi_sort])

['NT' 'LE' 'DE' 'EN' 'ES']


Q: en utilisant ce modèle probabiliste pour générer un mot de passe, quelle est l'entropie d'un mot de passe de 8 lettres ?

In [39]:
entropie_bi = 4 * entropie(freq_bi)
print(entropie_bi)

30.142264046464188


On remarque qu'on gange environ 1 bit et demi d'entropie par rapport au cas ou les 8 lettres sont tirées indépendaments. 

Q: Pourquoi cette entropie est-elle inférieure à celle du modèle construit sur des monogrammes ? Quelle propriété théorique de l'entropie peut justifier ce constat  ?

De manière qualitative, la première lettre contient maintenat de l'information sur la seconde. Il faut donc moins d'informations pour déterminer le mot de passe.
Le résultat théorique est que l'entropie mutuelle est inférieure à la somme des entropies avec cas d'égalité quand les v.a.r sont indépendantes. Dans la première question les var étaient indépendantes mais ici elles ne le sont plus.

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [69]:
t = time.time()

mdps = np.random.choice(letters_bi,size=(100000,8),p=freq_bi)

t_bi_100000 = time.time()-t
print(t_bi_100000, 's')


0.09486985206604004 s


Q: calculer le minorant de $G$ pour ce modèle

In [70]:
G_min_bi = 2**entropie_bi/4 + 1
print(G_min_bi)

296254956.70154107


Q: combien de temps cela prendra-t-il si l'on suppose qu'il est possible de prendre le générateur codé précédemment ? (en minutes)

In [72]:
print("La génération d'un bon mot de passe prend en moyenne basse", G_min_bi*t_bi_100000/6000000, "minutes")

La génération d'un bon mot de passe prend en moyenne basse 4.684277319351049 minutes


On remarque que le temps de calcul est divisé par plus de 2, ce qui est logique car le fait de diminuer l'entropie de plus de quasiment 1,5 bits revient à diviser le minorant de G par plus de 2.

Q: si maintenant on change de stratégie et on tire aléatoirement chaque lettre de l'alphabet de façon uniforme, quelle est l'entropie de ce nouveau générateur ?

In [84]:
freq_uni=np.ones(26)/26
entropie_uni = 8*entropie(freq_uni)

Q: A l'aide de la fonction `np.random.choice()`, calculer le temps nécessaire en secondes pour tirer 100 000 mots de passes en utilisant ce générateur ?

In [85]:
t = time.time()

mdps_uni = np.random.choice(letters_mono,size=(100000,8))

t_uni_100000 = time.time()-t
print(t_uni_100000, 's')

0.02785515785217285 s


Q: calculer le minorant de $G$ pour ce modèle

In [86]:
G_min_uni = 2**entropie_uni/4 + 1
print(G_min_uni)

52206766144.99937


Q: dans ce cas précis, quelle est la valeur exacte de $G$?

In [91]:
G_uni = 8**26/2
print('La valeur exacte de G est ici', G_uni)

La valeur exacte de G est ici 1.5111572745182865e+23


Il y a $8^{26}$ possibilités équiprobables, parmi lesquelles on teste des possibilités aléatoires, ce qui donne donc une espérance de $8^{26}/2$

Q: combien de temps cela prendra-t-il en utilisant le générateur codé précédemment ? (en minutes)

In [94]:
print("La génération d'un bon mot de passe prend en moyenne", G_uni*t_uni_100000/6000000, "minutes")

La génération d'un bon mot de passe prend en moyenne 701558740352769.5 minutes


Q: implémenter une attaque pratique qui consiste à:
1. **pour le défenseur:** tirer un mot de passe de 4 lettres consécutives à partir de ce texte de Victor Hugo (texteFrancais.txt) tiré des Misérables.  
2. **pour l'attaquant:** utiliser le modèle bigramme pour générer des mots de passe et minimiser le nombre d'essais. Pour cela on pourra :
    * dans un premier temps pré-calculer un **dictionnaire**, qui contriendra un nombre de MdP générés classés dans l'ordre du plus probable au moins probable et qui ne contient pas de doublons 
    * dans un deuxième temps appeler ce dictionnaire pour comparer chacune de ses entrées au mot de passe généré.
3. Il faudra faire ses tests plusieurs fois afin de d'obtenir un nombre moyens d'appel au dictionnaire nécessaire
4. Il sera intéressant de comparer le nombre trouvé à la valeur de G (qui est une borne inférieure)
5. Question annexe: Par un simple calcul, si le générateur utilisé n'est plus ce générateur mais un générateur qui tire chaque lettre de façon équiprobable, rappeler la valeur de G. Comparer cette valeur avec la valeur trouvée en utilisant la stratégie "des 4 lettres consécutives".

In [133]:
## Fonction générant un mot de passe
def get_passwd():
    text_hugo = open("texteFrancais.txt","r")
    str_hugo = str(text_hugo.read())

    # On remplace des lettres avec accent avec des lettres sans accent
    str_hugo = str_hugo.replace("Â", "A")
    str_hugo = str_hugo.replace("Ù", "U")
    str_hugo = str_hugo.replace("Ô", "O")
    size_txt = len(str_hugo)

    idx_rand = np.random.randint(size_txt-4)
    #print(idx_rand)

    psswd = str_hugo[idx_rand:idx_rand+4]
    return(psswd)

In [101]:
# Génération du dictionnaire
dict_passwd = dict()
for idx1,dbl1 in enumerate(letters_bi):
    for idx2,dbl2 in enumerate(letters_bi):
        dict_passwd[dbl1+dbl2] = freq_bi[idx1]*freq_bi[idx2]


In [127]:
dict_sorted = dict(sorted(dict_passwd.items(), key=lambda item: item[1], reverse = True))

In [146]:
# Attaques sur 1000 mots de passes

nb_trial = 1000
vec_nb_trials = np.zeros(nb_trial)
tab_passwd = np.array([get_passwd() for k in range (nb_trial)])
list_sorted = list(dict_sorted.keys())
tps =[]
for passwd in tab_passwd:
    print(passwd)
    found  = False
    i = 0
    while not found :
        test_passwd = list_sorted[i]
        i+=1
        if(test_passwd == passwd):
            found = True
    print(i)
    tps.append(i)

ANTU
6507
NTLA
238
IRQU
2110
MOUR
8118
ONGT
43762
ENNE
163
ITMM
6324
NSUR
1653
UPAI
7454
ECTI
2142
NGRA
16146
SEZP
67353
RIVE
10300
RSON
964
LLAR
5811
NTEA
3431
TBRU
89641
UPFA
31959
LFUT
49175
AFOR
31103
LAPR
1749
ITFA
5317
LDEV
49782
ONTA
591
ABEA
31037
PROP
24849
COMP
16343
RQUU
105806
SLEP
7667
TMCH
28723
ENTP
2108
VEQU
3366
TAGE
9914
NETA
1473
SEXE
34412
NTEL
206
IELP
33799
XNOB
206278
NNEE
25148
REET
578
SSAT
5219
LAIT
371
SEDE
152
ROFI
22178
LLES
782
FOIS
12968
ENMM
3913
DANS
5266
LLEB
37716
ETRE
577
IMPE
28314
NMEL
16592
RVAN
13094
ENFA
3157
ILET
1668
VEAU
9642
UNEP
4331
AMAT
13361
IMEL
9736
SUNO
15749
RENA
2236
DIST
6995
PRIS
1870
ESUB
9789
TEAF
14205
NOUV
27183
NECE
1568
BAPT
99637
NSQU
991
ESEN
3
PUIS
18724
LETA
399
EXAC
33038
SONN
14379
EUNE
2449
UDAN
12895
ESPR
342
REGA
15012
REVI
4801
NUFA
46887
SLES
1293
SUIT
3438
VECU
30926
UNEP
4331
NTSE
225
DEDI
1917
SOUV
20103
ONNE
353
AUVE
9641
SSAT
5219
ICIL
8226
LUIF
38713
SEDE
152
RQUI
30760
AIEN
35
TQUE
16253
VAIE
9041
OURE
253


In [147]:
print(np.mean(tps))

14867.429


Calcul de G min théorique :

In [149]:
entropie  =  2*entropie(freq_bi)
G_min = 2**entropie/4 + 1
print(G_min)

8607.029219412707


On voit donc que notre méthode n'est pas optimale 

## Conclusions 

- Définir des bonnes pratiques pour le défenseur, i.e. la personne cherchant à concevoir un système de génération de mots de passe ? 
- Définior des bonnes pratiques pour l'attaquant, i.e. la personne essayant de trouver le mot de passe ?

Les bonnes pratiques défenseur: 

    - avoir un long mot de passe
    - avoir un maximum de caractère différents (chiffres, majuscule, ponctuation, accents, ...)
    - ne pas prendre un mot issue de la langue courante, préférer une suite de lettres peu commune
    
Les bonnes pratiques attaquant :

    - commencer par les combinaisons les plus probables
    - avoir un ordianteur puissant...
    - bien connaitre sa cible ( notament sa langue)
    

**To do**

## Un peu de lecture
Cet article montre comment des hackers, à partir de leaks de bases de mots de passes, peuvent rapidement arriver à trouver le votre:
https://arstechnica.com/information-technology/2013/05/how-crackers-make-minced-meat-out-of-your-passwords/
